# 데이터 입력 함수

# 날짜별 - 인원수 기준 & 가중치


In [ ]:
song_sessions = {
    'Violet': {'보컬': '오지원','기타1' : '윤진경', '기타2': '오지원', '베이스':'신정현','드럼': '박수빈'},
    'ride': {'보컬': '곽민욱','기타1' : '손호유', '기타2': '곽민욱', '베이스':'신정현','드럼': '박수빈'}
}

from datetime import date, datetime, timedelta

persons_availability = {
    '오지원': [date(2025, 5, 13)],
    '곽민욱': [date(2025, 5, 15)],
    '손호유': [date(2025, 5, 13)],
    '윤진경': [date(2025, 5, 13),date(2025, 5, 15)],
    '박수빈': [date(2025, 5, 15)],
    '신정현': [date(2025, 5, 13)]
}

base_schedule = [date(2025, 5, 13), date(2025, 5, 14), date(2025, 5, 15)]

session_weight = {
    '드럼': 16,
    '기타1': 8,
    '보컬': 4,
    '베이스': 2,
    '기타2': 2,
    '건반': 1
}


'\n오지원 - 가능한시간, (세션,곡)\n\n1. 곡을 쓰고\n2. 세션 칸에 이름 적기\n3. 시간까지\n\n4. 시간의 교집합 개수 - len 길이로\n5. len 길이 같을 때 : 드럼(16) > 기타1(8) > 보컬(4) > 베이스(2)=기타2(2) > 건반(1)\n'

In [ ]:
person_role = {}

for song, sessions in song_sessions.items():
    for session, name in sessions.items():
        if name not in person_role:
            person_role[name] = []
        person_role[name].append((song, session))
print(person_role)

{'오지원': [('Violet', '보컬'), ('Violet', '기타2')], '윤진경': [('Violet', '기타1')], '신정현': [('Violet', '베이스'), ('ride', '베이스')], '박수빈': [('Violet', '드럼'), ('ride', '드럼')], '곽민욱': [('ride', '보컬'), ('ride', '기타2')], '손호유': [('ride', '기타1')]}


In [ ]:
# class Person 만들기
# 속성: name, role, available_time

class Person:
  def __init__(self, name):
    self.name = name
    self.available_time = persons_availability[name]
    self.role = person_role[name]

  def print_person(self):
    print(self.available_time)
    print(self.role)


people = []

for name, _ in person_role.items():
  if name not in people:
    people.append(Person(name))

for person in people:
  print(person.name)
  print(person.role)

오지원
[('Violet', '보컬'), ('Violet', '기타2')]
윤진경
[('Violet', '기타1')]
신정현
[('Violet', '베이스'), ('ride', '베이스')]
박수빈
[('Violet', '드럼'), ('ride', '드럼')]
곽민욱
[('ride', '보컬'), ('ride', '기타2')]
손호유
[('ride', '기타1')]


In [ ]:
from collections import defaultdict

def Date_Find():
  people_of_date = defaultdict(list)     # 공통되는 날짜 찾기

  for date in base_schedule:
    people_of_date[date] = []
    for person in people:
      if date in person.available_time:
        people_of_date[date].append((person.name, person.role))     # 만약에 여기서 시간에 해당하는 이름만 뽑고 싶다면 append(person.name)으로 수정

  max_len = max(len(amount) for amount in people_of_date.values())
  fixed_date = [date for date, amount in people_of_date.items() if len(amount) == max_len]

  people_of_date = {date: val for date, val in people_of_date.items() if date in fixed_date}    # 전체 people_of_date에서 공통되는 시간들만 추출하기

  return people_of_date


'''
  people_of_date = {
    '2025-05-13': [('오지원', [('Violet', '보컬'), ('Violet', '세컨기타')]),
                   ('윤진경', [('Violet', '리드기타')]),
                   ('신정현', [('Violet', '베이스'), ('ride', '베이스')]),
                   ('손호유', [('ride', '리드기타')])],
    '2025-05-15': [('오지원', [('Violet', '보컬'), ('Violet', '세컨기타')]),
                   ('윤진경', [('Violet', '리드기타')]),
                   ('박수빈', [('Violet', '드럼'), ('ride', '드럼')]),
                   ('곽민욱', [('ride', '보컬'), ('ride', '세컨기타')])]
  }
'''



Date_Find()

{datetime.date(2025, 5, 13): [('오지원', [('Violet', '보컬'), ('Violet', '기타2')]),
  ('윤진경', [('Violet', '기타1')]),
  ('신정현', [('Violet', '베이스'), ('ride', '베이스')]),
  ('손호유', [('ride', '기타1')])]}

In [ ]:
def Date_Find_weighted():
  people_of_date = defaultdict(list)

  for date in base_schedule:
        for person in people:
            if date in person.available_time:
                people_of_date[date].append((person.name, person.role))

  # 날짜별 가중치 계산
  weight_by_date = {
        date: sum(
            session_weight.get(role, 0)
            for _, roles in val
            for _, role in roles
            )
        for date, val in people_of_date.items()
    }

  # (날짜, 가중치) 튜플 리스트를 가중치 기준으로 내림차순 정렬
  sorted_by_weight = sorted(weight_by_date.items(), key=lambda x: x[1], reverse=True)

  return sorted_by_weight


Date_Find_weighted()

[(datetime.date(2025, 5, 15), 46), (datetime.date(2025, 5, 13), 26)]

# 타임테이블 - 인원수 기준

In [ ]:
"""
2025-05-13 오지의 제안:
합주할 시간 정하는 함수 외에,
합주할 시간을 정한 후 합주곡 타임테이블을 짜는 함수도 만들면 좋을 듯

1번째 함수: 합주할 시간을 정하는 함수
2번째 함수: 합주할 시간을 정한 후 합주곡 타임테이블을 짜는 함수
3번째 함수: 여러 개의 합주실을 빌렸을 때 합주곡 타임테이블 짜기
"""

'\n2025-05-13 오지의 제안:\n합주할 시간 정하는 함수 외에,\n합주할 시간을 정한 후 합주곡 타임테이블을 짜는 함수도 만들면 좋을 듯\n\n1번째 함수: 합주할 시간을 정하는 함수\n2번째 함수: 합주할 시간을 정한 후 합주곡 타임테이블을 짜는 함수\n3번째 함수: 여러 개의 합주실을 빌렸을 때 합주곡 타임테이블 짜기\n'

In [ ]:
song_sessions = {
    'Violet': {'보컬': '오지원','기타1' : '윤진경', '기타2': '오지원', '베이스':'신정현','드럼': '박수빈'},
    'ride': {'보컬': '곽민욱','기타1' : '손호유', '기타2': '곽민욱', '베이스':'신정현','드럼': '박수빈'}
}


session_weight = {
    '드럼': 16,
    '기타1': 8,
    '보컬': 4,
    '베이스': 2,
    '기타2': 2,
    '건반': 1
}


In [ ]:
# songs_sessions를 person_role로 바꿈

person_role = {}

for song, sessions in song_sessions.items():
    for session, name in sessions.items():
        if name not in person_role:
            person_role[name] = []
        person_role[name].append((song, session))


print(person_role)

{'오지원': [('Violet', '보컬'), ('Violet', '기타2')], '윤진경': [('Violet', '기타1')], '신정현': [('Violet', '베이스'), ('ride', '베이스')], '박수빈': [('Violet', '드럼'), ('ride', '드럼')], '곽민욱': [('ride', '보컬'), ('ride', '기타2')], '손호유': [('ride', '기타1')]}


In [ ]:
# datetime 객체 이용한 예시 데이터 (시간은 30분 단위)
# 나중에 이 데이터들을 만드는 함수도 짜야 함 (프론트엔드)
# Streamlit (파이썬 도구) 쓰면 될듯


from datetime import datetime, timedelta

persons_availability = {
    '오지원': [
        datetime(2025, 5, 13, 17, 00),
        datetime(2025, 5, 13, 17, 30),
        datetime(2025, 5, 13, 18, 00),
        datetime(2025, 5, 13, 18, 30),
        datetime(2025, 5, 13, 19, 00),
        datetime(2025, 5, 13, 19, 30),
    ],
    '곽민욱': [
        datetime(2025, 5, 15, 18, 00),
        datetime(2025, 5, 15, 18, 30),
        datetime(2025, 5, 15, 19, 00),
        datetime(2025, 5, 15, 19, 30),

    ],
    '손호유': [
        datetime(2025, 5, 13, 18, 00),
        datetime(2025, 5, 13, 18, 30),
        datetime(2025, 5, 13, 19, 00),
        datetime(2025, 5, 13, 19, 30),
    ],
    '윤진경': [
        datetime(2025, 5, 13, 18, 00),
        datetime(2025, 5, 13, 18, 30),
        datetime(2025, 5, 13, 19, 00),
        datetime(2025, 5, 13, 19, 30),
        datetime(2025, 5, 15, 18, 00),
        datetime(2025, 5, 15, 18, 30),
        datetime(2025, 5, 15, 19, 00),
        datetime(2025, 5, 15, 19, 30),
    ],
    '박수빈': [
        datetime(2025, 5, 15, 17, 00),
        datetime(2025, 5, 15, 17, 30),
        datetime(2025, 5, 15, 18, 00),
        datetime(2025, 5, 15, 18, 30),
        datetime(2025, 5, 15, 19, 00),
        datetime(2025, 5, 15, 19, 30),
    ],
    '신정현': [
        datetime(2025, 5, 13, 18, 00),
        datetime(2025, 5, 13, 18, 30),
        datetime(2025, 5, 13, 19, 00),
        datetime(2025, 5, 13, 19, 30),
        datetime(2025, 5, 13, 20, 00),
        datetime(2025, 5, 13, 20, 30),
    ]
}

base_schedule = [
    datetime(2025, 5, 13, 17, 00),
    datetime(2025, 5, 13, 18, 30),
    datetime(2025, 5, 13, 18, 00),
    datetime(2025, 5, 13, 18, 30),
    datetime(2025, 5, 13, 19, 00),
    datetime(2025, 5, 13, 19, 30),
    datetime(2025, 5, 13, 20, 00),
    datetime(2025, 5, 13, 20, 30),
    datetime(2025, 5, 15, 17, 00),
    datetime(2025, 5, 15, 17, 30),
    datetime(2025, 5, 15, 18, 00),
    datetime(2025, 5, 15, 18, 30),
    datetime(2025, 5, 15, 19, 00),
    datetime(2025, 5, 15, 19, 30),
]

#base_schedule의 시작하는 시간이랑 끝나는 시간 정한담에 30분 단위로 추가 될 수 있게 하는 반복문으로 해도 될 듯

In [ ]:
# class Person 만들기
# 속성: name, role, available_time

class Person:
  def __init__(self, name):
    self.name = name
    self.available_time = persons_availability[name]
    self.role = person_role[name]

  def print_person(self):
    print(self.available_time)
    print(self.role)


people = []

for name, _ in person_role.items():
  if name not in people:
    people.append(Person(name))

for person in people:
  print(person.name)
  print(person.role)

오지원
[('Violet', '보컬'), ('Violet', '기타2')]
윤진경
[('Violet', '기타1')]
신정현
[('Violet', '베이스'), ('ride', '베이스')]
박수빈
[('Violet', '드럼'), ('ride', '드럼')]
곽민욱
[('ride', '보컬'), ('ride', '기타2')]
손호유
[('ride', '기타1')]


In [ ]:
# 곡별로 공통되는 시간대 찾기
# (시간, 합주곡) 튜플 이용해서 표현하기

from collections import defaultdict

def Time_Find():

  people_of_time = {}     # 공통되는 날짜 찾기

  for time in base_schedule:
    people_of_time[time] = []

    for person in people:
      if time in person.available_time:
        for _ in person.role:
          people_of_time[time].append(_)  # 시간에 해당하는 (곡, 세션) 튜플 추가

  time_song_counts = {}

  for time, _ in people_of_time.items():
      song_count = defaultdict(int)

      for song, role in _:
          song_count[song] += 1

      time_song_counts[time] = dict(song_count)

  return time_song_counts

Time_Find()

{datetime.datetime(2025, 5, 13, 17, 0): {'Violet': 2},
 datetime.datetime(2025, 5, 13, 18, 30): {'Violet': 4, 'ride': 2},
 datetime.datetime(2025, 5, 13, 18, 0): {'Violet': 4, 'ride': 2},
 datetime.datetime(2025, 5, 13, 19, 0): {'Violet': 4, 'ride': 2},
 datetime.datetime(2025, 5, 13, 19, 30): {'Violet': 4, 'ride': 2},
 datetime.datetime(2025, 5, 13, 20, 0): {'Violet': 1, 'ride': 1},
 datetime.datetime(2025, 5, 13, 20, 30): {'Violet': 1, 'ride': 1},
 datetime.datetime(2025, 5, 15, 17, 0): {'Violet': 1, 'ride': 1},
 datetime.datetime(2025, 5, 15, 17, 30): {'Violet': 1, 'ride': 1},
 datetime.datetime(2025, 5, 15, 18, 0): {'Violet': 2, 'ride': 3},
 datetime.datetime(2025, 5, 15, 18, 30): {'Violet': 2, 'ride': 3},
 datetime.datetime(2025, 5, 15, 19, 0): {'Violet': 2, 'ride': 3},
 datetime.datetime(2025, 5, 15, 19, 30): {'Violet': 2, 'ride': 3}}

# 타임테이블 - 인원수 기준 & 세션별 가중치